In [34]:
import os
import pandas as pd

MAC_TRAIN_PATH = 'dataset/20news-bydate-train/comp.sys.mac.hardware/'
PC_TRAIN_PATH = 'dataset/20news-bydate-train/comp.sys.ibm.pc.hardware/'
MAC_TEST_PATH = 'dataset/20news-bydate-test/comp.sys.mac.hardware/'
PC_TEST_PATH = 'dataset/20news-bydate-test/comp.sys.ibm.pc.hardware/'


def getDataFrame(path):
    """
    Args:
        path: the path of a directory of files, string.
    Returns:
        A dataframe of all the content of the files in the given path, 
        datframe index is the filename. 
    """

    data = []
    filenames = os.listdir(path)
    for name in filenames:
        document = path+name
        data.append(open(document, "rb").read())
    df = pd.DataFrame(data, index = filenames, columns = ['Data'])
    return df


mac_train = getDataFrame(MAC_TRAIN_PATH)
pc_train = getDataFrame(PC_TRAIN_PATH)
mac_test = getDataFrame(MAC_TEST_PATH)
pc_test = getDataFrame(PC_TEST_PATH)

In [38]:
mac_train['Data'][0]

b"From: walsteyn@fys.ruu.nl (Fred Walsteijn)\nSubject: Re: built-in video problems on Mac IIsi !!??!!\nOrganization: Physics Department, University of Utrecht, The Netherlands\nLines: 24\n\nIn <1993Apr19.164611.1018@fys.ruu.nl> walsteyn@fys.ruu.nl (Fred Walsteijn) writes:\n\n>I've seen the following problem om three Mac IIsi machines\n>all with 17 Mb RAM installed (70 or 80 ns SIMMs).\n\n>If the contents of a window are being calculated and updated\n>a lot of strange horizontal lines are temporarily generated\n>on the screen.  The lines translate to the top of the screen and\n>have a slightly lower brightness than their surroundings (they\n>are a few millimeters apart).\n>I admit that they are vague, but they can still be distinguished clearly,\n>especially if the environment (i.e. the rest of the room) is a bit dark.\n>Applications which produce this effect are:\n>- the previewer of DirectTeX 1.2 (i.e. DVIReader 1.2)\n>- Kaleidagraph 2.1.1/FPU\n\nAnother program which produces this ef

In [36]:
len(mac_train)

578